## import modules

In [18]:
import pandas as pd
from scipy import stats
import numpy as np

## census data processing

In [19]:
# for population data
cols=['State', 'Level', 'Name', 'TRU', 'No_HH', 'TOT_P', 'TOT_M', 'TOT_F']
census=pd.read_excel('datasets/census.xlsx',engine='openpyxl',usecols=cols)
censusIndia=census.iloc[0:3,:]
census=census.loc[census.Level=='STATE']
census=census.append(censusIndia,ignore_index=True)
census.sort_values(by=['State'],axis=0,inplace=True)
census.reset_index(drop=True,inplace=True)

## urban-rural

In [20]:
c18=pd.read_excel('datasets/C-18.xlsx',engine='openpyxl',skiprows=6,header=None)

In [21]:
STATE_NAMES=[]
for state in c18.iloc[:,2].values:
    if not (state in STATE_NAMES):
        STATE_NAMES.append(state)

In [31]:
useful_data=[]
for i,state in enumerate(STATE_NAMES):
    
    # here i is the state code
    urban_pop=census[(census['State']==i) & (census['TRU']=='Urban')]['TOT_P'].values[0]
    rural_pop=census[(census['State']==i) & (census['TRU']=='Rural')]['TOT_P'].values[0]
    
    # tri
    tri_urban=c18[(c18.iloc[:,0]==i) & (c18.iloc[:,4]=='Total') & (c18.iloc[:,3]=='Urban')].iloc[0,8]
    tri_rural=c18[(c18.iloc[:,0]==i) & (c18.iloc[:,4]=='Total') & (c18.iloc[:,3]=='Rural')].iloc[0,8]

    #bi
    bi_urban=c18[(c18.iloc[:,0]==i) & (c18.iloc[:,4]=='Total') & (c18.iloc[:,3]=='Urban')].iloc[0,5] - tri_urban
    bi_rural=c18[(c18.iloc[:,0]==i) & (c18.iloc[:,4]=='Total') & (c18.iloc[:,3]=='Rural')].iloc[0,5] - tri_rural

    #uni
    uni_urban=urban_pop-bi_urban-tri_urban
    uni_rural=rural_pop-bi_rural-tri_rural

    '''
    item={
        'state-code':i,
        'state-name':state,
        'urban-pop':urban_pop,
        'rural-pop':rural_pop,
        'tri-urban-ratio':tri_urban/urban_pop,
        'tri-rural-ratio':tri_rural/rural_pop,
        'bi-urban-ratio':bi_urban/urban_pop,
        'bi-rural-ratio':bi_rural/rural_pop,
        'uni-urban-ratio':uni_urban/urban_pop,
        'uni-rural-ratio':uni_rural/rural_pop
    }
    '''
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html   
    p_value=stats.ttest_ind([urban_pop/rural_pop]*3,[tri_urban/tri_rural,bi_urban/bi_rural,uni_urban/uni_rural]).pvalue
    
    item={
        'state-code':i,
        'state-name':state,
        'p-value':p_value,
        'urban-to-rural':urban_pop/rural_pop,
        'tri-urban-to-rural-ratio':tri_urban/tri_rural,
        'bi-urban-to-rural-ratio':bi_urban/bi_rural,
        'uni-urban-to-rural-ratio':uni_urban/uni_rural
    }

    useful_data.append(item)

In [32]:
df=pd.DataFrame(useful_data)

In [33]:
df

,state-code,state-name,p-value,urban-to-rural,tri-urban-to-rural-ratio,bi-urban-to-rural-ratio,uni-urban-to-rural-ratio
0,0,INDIA,0.273554,0.452302,1.430749,0.799338,0.334907
1,1,JAMMU & KASHMIR,0.536400,0.376945,0.665353,0.405175,0.286853
2,2,HIMACHAL PRADESH,0.215755,0.111487,0.236699,0.277890,0.082272
3,3,PUNJAB,0.486332,0.599575,0.850663,0.824946,0.433398
4,4,CHANDIGARH,0.561803,35.406126,72.967149,33.998779,26.538221
5,5,UTTARAKHAND,0.198847,0.433332,1.659954,1.053221,0.342939
6,6,HARYANA,0.206276,0.535581,1.079289,1.030821,0.434291
7,7,NCT OF DELHI,0.347604,39.062669,247.124406,57.548258,30.126520
8,8,RAJASTHAN,0.169959,0.331029,0.651746,0.777705,0.292507
9,9,UTTAR PRADESH,0.149790,0.286479,1.039412,1.102367,0.225343


- for no state or ut the ratio is significantly different
- should I try the t-test on normalized vectors(that is they have unit length) 
- could try out the same thing with different values 